In [13]:
from pymongo import MongoClient

client = MongoClient('127.0.0.1', 27017)
client = MongoClient('mongodb://127.0.0.1:27017/')
db = client.movielens

In [43]:
from pymongo import MongoClient
from datetime import datetime

# MongoDB 연결
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']

# 인덱스 생성 시간 측정 함수
def measure_execution_time(query_func):
    start_time = datetime.now()
    query_func()
    end_time = datetime.now()
    return (end_time - start_time).total_seconds() * 1000  # milliseconds

# 인덱스 생성 함수
def create_index(collection, index_spec):
    if index_spec is not None:
        collection.create_index(index_spec)

# 인덱스 삭제 함수
def drop_all_indexes(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop_indexes()

# 쿼리 및 인덱스 설정
queries = [
    # Question #2
    {
        'pre_drop': lambda: drop_all_indexes(db),
        'pre_query': lambda: db.ml_tags.find({'tag': "italian western"}).sort('title', 1).distinct('title'),
        'index1': ('ml_tags', 'tag'),
        'index2': ('ml_movies', 'movieId'),
        'post_query': lambda: db.ml_tags.find({'tag': "italian western"}).sort('title', 1).distinct('title'),
    },
    # Question #3
    {
        'pre_drop': lambda: drop_all_indexes(db),
        'pre_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'title': "Interstellar (2014)"}},
            {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
        ]),
        'index1': ('ml_movies', 'title'),
        'index2': ('ml_ratings', 'movieId'),
        'post_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'title': "Interstellar (2014)"}},
            {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
        ]),
    },
    # Question #4
    {
        'pre_drop': lambda: drop_all_indexes(db),
        'pre_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'userId': 54313}},
            {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
            {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
        ]),
        'index1': ('ml_ratings', 'userId'),
        'index2': ('ml_ratings', 'movieId'),
        'post_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'userId': 54313}},
            {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
            {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
        ]),
    },
]

# 결과 저장할 딕셔너리
results = {}

# 각 쿼리 실행 및 결과 저장
for i, query in enumerate(queries, start=1):
    result = {}
    pre_drop_time = measure_execution_time(query['pre_drop'])
    result['생성 전'] = pre_drop_time
    create_index(db[query['index1'][0]], query['index1'][1])
    create_index(db[query['index2'][0]], query['index2'][1])
    pre_query_time = measure_execution_time(query['pre_query'])
    result['생성 후'] = pre_query_time
    result['차이'] = pre_query_time - pre_drop_time
    results[f'Question #{i}'] = result

# 결과 출력
for question, times in results.items():
    print(f"{question}:")
    print(f"생성 전: {times['생성 전']:.3f}ms")
    print(f"생성 후: {times['생성 후']:.3f}ms")
    print(f"차이: {times['차이']:.3f}ms")
    print()

# 차이의 평균 계산 및 출력
diff_avg = sum(result['차이'] for result in results.values()) / len(results)
print(f"평균 차이: {diff_avg:.3f} ms")

Question #1:
생성 전: 29.729ms
생성 후: 5.025ms
차이: -24.704ms

Question #2:
생성 전: 4.002ms
생성 후: 11343.145ms
차이: 11339.143ms

Question #3:
생성 전: 13.636ms
생성 후: 7.963ms
차이: -5.673ms

평균 차이: 3769.589 ms


In [6]:

for collection_name in db.list_collection_names():
    collection = db[collection_name]
    collection.drop_indexes()

import time

start = time.time()

avg = db.ml_ratings.aggregate([
    {'$match': {'title': "Interstellar (2014)"}},
    {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}])

end = time.time()

diff = (end - start)





In [28]:
# # 인덱스 생성 함수
# def create_index(collection, index_spec):
#     if index_spec is not None:
#         collection.create_index(index_spec)


db.ml_movies.create_index('title')
db.ml_ratings.create_index('movieId')



'movieId_1'

In [21]:
for collection_name in db.list_collection_names():
    collection = db[collection_name]
    collection.drop_indexes()

In [27]:
# 'ratings' 컬렉션 삭제
db.ratings.drop()

# 'movies' 컬렉션 삭제
db.movies.drop()

In [26]:
# 데이터베이스 내의 모든 컬렉션 이름 조회
collection_names = db.list_collection_names()

# 각 컬렉션에 대한 인덱스 정보 출력
for collection_name in collection_names:
    collection = db[collection_name]
    print(f"인덱스 목록 - {collection_name}:")
    for index in collection.list_indexes():
        print(index)

인덱스 목록 - ml_movies:
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'movielens.ml_movies')])
인덱스 목록 - ml_tags:
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'movielens.ml_tags')])
인덱스 목록 - ml_ratings:
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'movielens.ml_ratings')])
인덱스 목록 - ratings:
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'movielens.ratings')])
SON([('v', 2), ('key', SON([('movieId', 1)])), ('name', 'movieId_1'), ('ns', 'movielens.ratings')])
인덱스 목록 - movies:
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'movielens.movies')])
SON([('v', 2), ('key', SON([('title', 1)])), ('name', 'title_1'), ('ns', 'movielens.movies')])


In [32]:
from pymongo import MongoClient
import time

# MongoDB 연결
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']

# 인덱스 생성 시간 측정 함수
def measure_execution_time(query_func):
    start_time = time.time()
    query_func()
    end_time = time.time()
    return end_time - start_time

# 인덱스 생성 함수
def create_index(collection, index_spec):
    if index_spec is not None:
        collection.create_index(index_spec)

# 인덱스 삭제 함수
def drop_all_indexes(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop_indexes()

# 쿼리 및 인덱스 설정
queries = [
    {
    'query': lambda: list(set(
        db.ml_movies.find(
            {"movieId": {"$in": [doc['movieId'] for doc in db.ml_tags.find({"tag": "italian western"}, {"movieId": 1})]}},
            {"title": 1, "_id": 0}
        ).distinct("title")
    )),
    }
    ,
    {
        'query': lambda: db.ml_ratings.aggregate([
            {'$match': {'title': "Interstellar (2014)"}},
            {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
        ]),
    },
    {
        'query': lambda: db.ml_ratings.aggregate([
            {'$match': {'userId': 54313}},
            {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
            {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
        ]),
    },
]

# 모든 인덱스 삭제
# drop_all_indexes(db)

# 결과 저장할 딕셔너리
results = {}

# 생성 전 쿼리 실행
for i, query in enumerate(queries, start=1):
    pre_query_time = measure_execution_time(query['query'])
    results[f'Question #{i+1}'] = {'생성 전': pre_query_time}

# 중복되지 않는 인덱스 생성
# create_index(db.ml_tags, [('tag', 1)])
# create_index(db.ml_movies, [('movieId', 1), ('title', 1)])
# create_index(db.ml_ratings, [('movieId', 1), ('userId', 1)])

# 생성 후 쿼리 실행
for i, query in enumerate(queries, start=1):
    post_query_time = measure_execution_time(query['query'])
    results[f'Question #{i+1}']['생성 후'] = post_query_time
    results[f'Question #{i+1}']['차이'] = results[f'Question #{i+1}']['생성 후'] - results[f'Question #{i+1}']['생성 전']

# 결과 출력
for question, times in results.items():
    print(f"{question}:")
    print(f"생성 전: {times['생성 전']:.3f}ms")
    print(f"생성 후: {times['생성 후']:.3f}ms")
    print(f"차이: {times['차이']:.3f}ms")
    print()

# 차이의 평균 계산 및 출력
diff_avg = sum(result['차이'] for result in results.values()) / len(results)
print(f"평균 차이: {diff_avg:.3f} ms")



Question #2:
생성 전: 0.023ms
생성 후: 0.002ms
차이: -0.021ms

Question #3:
생성 전: 10.877ms
생성 후: 10.725ms
차이: -0.152ms

Question #4:
생성 전: 10.809ms
생성 후: 10.649ms
차이: -0.160ms

평균 차이: -0.111 ms


In [31]:
create_index(db.ml_tags, [('tag', 1)])
create_index(db.ml_movies, [('movieId', 1), ('title', 1)])
create_index(db.ml_ratings, [('movieId', 1), ('userId', 1)])

In [45]:
from pymongo import MongoClient
from datetime import datetime

# MongoDB 연결
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']

# 인덱스 생성 시간 측정 함수
def measure_execution_time(query_func):
    start_time = datetime.now()
    query_func()
    end_time = datetime.now()
    return (end_time - start_time).total_seconds() * 1000  # milliseconds

# 인덱스 생성 함수
def create_index(collection, index_spec):
    if index_spec is not None:
        collection.create_index(index_spec)

# 인덱스 삭제 함수
def drop_all_indexes(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop_indexes()

# 쿼리 및 인덱스 설정
queries = [
    # Question #2
    {
        'pre_drop': lambda: drop_all_indexes(db),
        'pre_query': lambda: db.ml_tags.find({'tag': "italian western"}).sort('title', 1).distinct('title'),
        'index1': ('ml_tags', 'tag'),
        'index2': ('ml_movies', 'movieId'),
        'post_query': lambda: db.ml_tags.find({'tag': "italian western"}).sort('title', 1).distinct('title'),
    },
    # Question #3
    {
        'pre_drop': lambda: drop_all_indexes(db),
        'pre_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'title': "Interstellar (2014)"}},
            {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
        ]),
        'index1': ('ml_movies', 'title'),
        'index2': ('ml_ratings', 'movieId'),
        'post_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'title': "Interstellar (2014)"}},
            {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
        ]),
    },
    # Question #4
    {
        'pre_drop': lambda: drop_all_indexes(db),
        'pre_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'userId': 54313}},
            {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
            {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
        ]),
        'index1': ('ml_ratings', 'userId'),
        'index2': ('ml_ratings', 'movieId'),
        'post_query': lambda: db.ml_ratings.aggregate([
            {'$match': {'userId': 54313}},
            {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
            {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
        ]),
    },
]

# 결과 저장할 딕셔너리
results = {}

# 각 쿼리 실행 및 결과 저장
for i, query in enumerate(queries, start=1):
    result = {}
    pre_drop_time = measure_execution_time(query['pre_drop'])
    result['생성 전'] = pre_drop_time
    create_index(db[query['index1'][0]], query['index1'][1])
    create_index(db[query['index2'][0]], query['index2'][1])
    pre_query_time = measure_execution_time(query['pre_query'])
    result['생성 후'] = pre_query_time
    result['차이'] = pre_query_time - pre_drop_time
    results[f'Question #{i}'] = result

# 결과 출력
for question, times in results.items():
    print(f"{question}:")
    print(f"생성 전: {times['생성 전']:.3f}ms")
    print(f"생성 후: {times['생성 후']:.3f}ms")
    print(f"차이: {times['차이']:.3f}ms")
    print()

# 차이의 평균 계산 및 출력
diff_avg = sum(result['차이'] for result in results.values()) / len(results)
print(f"평균 차이: {diff_avg:.3f} ms")

In [ ]:
newjeans.delete_many({})

In [ ]:
newjeans.insert_one({'name':'Minji', 'age':18})

In [ ]:
pprint(list(newjeans.find()))

In [ ]:
newjeans.insert_many([
    {
        'name':'Hanni',
        'age': 19
    },
    {
        'name': 'Danielle',
        'age': 18
    },
    {
        'name': 'Haerin',
        'age': 17
    }
])

In [ ]:
pprint(list(newjeans.find()))

In [ ]:
newjeans.update_one(
    { 'name': 'Hannie' },
    { '$set': { 'location' : 'Rome' } },
    upsert=True
)

In [ ]:
pprint(list(newjeans.find()))

In [ ]:
newjeans.update_many(
    { 'age': 19 },
    { '$set': { 'location' : 'Paris' } },
    upsert=True
)

In [ ]:
pprint(list(newjeans.find()))

In [ ]:
newjeans.update_one(
    { 'name': 'Hyein' },
    { '$set': { 'age' : '15' } },
    upsert=True
)

In [ ]:
pprint(list(newjeans.find()))

In [ ]:
newjeans.delete_many({'location':'Paris'})

In [ ]:
pprint(list(newjeans.find()))

In [ ]:
db = client.lab  
col = db.imdb

In [ ]:
def input_movie_info():
    title = input("Movie title: ")
    director = input("Director: ")
    genre = input("Genre: ").split(",")
    score = float(input("Score: "))

    return title, director, genre, score

In [ ]:
t, d, g, s = input_movie_info()

col.insert_one(
    {
        'title':t,
        'director': d,
        'genre': g,
        'score': s
    }
)

print("[INFO] Success to insert!")

In [ ]:
pprint(list(col.find()))

In [ ]:
def input_movie_info():
    title = input("Movie title: ")
    director = input("Director: ")
    genre = input("Genre: ").split(",")
    score = float(input("Score: "))

    return title, director, genre, score

In [ ]:
quit = False
movies = []

while(not quit):
    t, d, g, s = input_movie_info()
    doc = {'title': t, 'director': d, 'genre': g, 'score': s }
    movies.append(doc)

    end = input("Quit? (Y/N)")

    if end =="Y":
        quit = True

col.insert_many(movies) 

print("[INFO] Success to insert!")

In [ ]:
pprint(list(col.find()))

In [ ]:
title = input("title")
director = input("director")
genre = input("genre").split(",")
score = float(input("score"))

col.update_one({'Movie Title':title}, {'$set' : {'Director':director, 'Genre': genre, 'score': score}})

In [ ]:
pprint(list(col.find()))

In [ ]:
print("[INFO] Enter the movie title you want to update")

t, d, g, s = input_movie_info()

col.update_one(
    {
        'title':t
    },
    {
        '$set': {
            'director': d,
            'genre': g,
            'score': s
        }
    }
)

print("[INFO] Success to update!")

In [ ]:
pprint(list(col.find()))

In [ ]:
print("[INFO] Enter the movie title you want to delete.")
title = input("Movie title: ")
delete_op = input("Are you sure? (Y/N)")

if delete_op == "Y":
    col.delete_many({
        'title': title
    })
    print("[INFO] Deleted")

In [ ]:
pprint(list(col.find()))

In [15]:
# 각 질문에 대한 쿼리 및 인덱스 생성
queries = [
    # Question #2: 인덱스 생성 전
    ("Question #2: 인덱스 생성 전", lambda: db.ml_tags.find({'tag': 'American Civil War'}).sort('title', 1).distinct('title'), None),

    # 인덱스 생성 (1번째 인덱스: 'tag' 필드)
    ("인덱스 생성", lambda: create_index(db.ml_tags, [('tag', 1)]), None),

    # 인덱스 추가 (2번째 인덱스: 'title' 필드)
    ("인덱스 추가", lambda: create_index(db.ml_tags, [('title', 1)]), None),

    # Question #2: 인덱스 생성 후 (3번째 인덱스: 'title' 필드)
    ("Question #2: 인덱스 생성 후", lambda: db.ml_tags.find({'tag': 'American Civil War'}).sort('title', 1).distinct('title'), [('title', 1)]),

    # 인덱스 삭제
    ("인덱스 삭제", lambda: drop_indexes(db.ml_tags), None),

    # Question #3: 인덱스 생성 전
    ("Question #3: 인덱스 생성 전", lambda: db.ml_ratings.aggregate([
        {'$match': {'title': 'Abraham Lincoln: Vampire Hunter (2012)'}},
        {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
    ]), None),

    # 인덱스 생성 (1번째 인덱스: 'title' 필드)
    ("인덱스 생성", lambda: create_index(db.ml_ratings, [('title', 1)]), None),

    # 인덱스 추가 (2번째 인덱스: 'rating' 필드)
    ("인덱스 추가", lambda: create_index(db.ml_ratings, [('rating', 1)]), None),

    # Question #3: 인덱스 생성 후 (3번째 인덱스: 'rating' 필드)
    ("Question #3: 인덱스 생성 후", lambda: db.ml_ratings.aggregate([
        {'$match': {'title': 'Abraham Lincoln: Vampire Hunter (2012)'}},
        {'$group': {'_id': '$title', 'averageRating': {'$avg': '$rating'}}}
    ]), [('rating', 1)]),

    # 인덱스 삭제
    ("인덱스 삭제", lambda: drop_indexes(db.ml_ratings), None),

    # Question #4: 인덱스 생성 전
    ("Question #4: 인덱스 생성 전", lambda: db.ml_ratings.aggregate([
        {'$match': {'userId': 2}},
        {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
        {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
    ]), None),

    # 인덱스 생성 (1번째 인덱스: 'userId' 필드)
    ("인덱스 생성", lambda: create_index(db.ml_ratings, [('userId', 1)]), None),

    # 인덱스 추가 (2번째 인덱스: 'rating' 필드)
    ("인덱스 추가", lambda: create_index(db.ml_ratings, [('rating', 1)]), None),

    # Question #4: 인덱스 생성 후 (3번째 인덱스: 'rating' 필드)
    ("Question #4: 인덱스 생성 후", lambda: db.ml_ratings.aggregate([
        {'$match': {'userId': 2}},
        {'$group': {'_id': None, 'totalRatings': {'$sum': 1}, 'totalRatingScores': {'$sum': '$rating'}, 'avgRatingOverall': {'$avg': '$rating'}}},
        {'$project': {'_id': 0, 'bias': {'$divide': [{'$subtract': ['$totalRatingScores', {'$multiply': ['$avgRatingOverall', '$totalRatings']}]}, '$totalRatings']}}}
    ]), [('rating', 1)]),
]
# 결과 저장할 딕셔너리
results = {}

# 10번 반복
for _ in range(1000):
    # 각 쿼리를 차례대로 실행하고 실행 시간 출력
    for i in range(0, len(queries), 5):  # 인덱스 추가 후 인덱스 생성 후 질문까지 처리하는 부분을 추가하여 index를 5씩 증가시킴
        query_name = queries[i][0]
        before_query_func = queries[i+1][1]
        after_query_func = queries[i+3][1]  # 수정된 부분: 인덱스 생성 후 쿼리 함수의 인덱스를 올바르게 설정

        # 인덱스 생성 전 쿼리 실행 및 시간 측정
        before_execution_time = measure_index_creation_time(before_query_func)

        # 인덱스 생성
        create_index(db[query_name], queries[i+1][2])

        # 인덱스 추가
        create_index(db[query_name], queries[i+2][2])

        # 인덱스 생성 후 쿼리 실행 및 시간 측정
        after_execution_time = measure_index_creation_time(after_query_func)

        # 차이 계산
        diff = round(after_execution_time - before_execution_time, 3)  # 수정된 부분: 차이 값을 절대값으로 변경

        # 결과 저장
        if query_name not in results:
            results[query_name] = {"생성 전": [], "생성 후": [], "차이": []}

        results[query_name]["생성 전"].append(before_execution_time)
        results[query_name]["생성 후"].append(after_execution_time)
        results[query_name]["차이"].append(diff)

# 평균값 계산 및 결과 출력
for query_name, result in results.items():
    print(query_name.split(':')[0] + ":")
    for key, value in result.items():
        if key == "차이":
            print(f"{key}: {round(sum(value) / len(value), 3)}ms")
        else:
            print(f"{key}: {value[0]:.3f}ms")
    print()

# 각 question의 차이 값 출력
average_diff_mean = {f"#{i+2}의 차이": round(sum(result['차이']) / len(result['차이']), 3) for i, result in enumerate(results.values())}
print(", ".join(average_diff_mean.keys()) + " 평균:", round(sum(average_diff_mean.values()) / len(average_diff_mean), 3), "ms")


NameError: name 'measure_index_creation_time' is not defined

In [5]:
from pymongo import MongoClient
import time

# MongoDB 연결 설정
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']  # 데이터베이스 선택

# 인덱스 생성 함수
def create_index(collection, index_spec):
    if index_spec is not None:
        collection.create_index(index_spec)

# 인덱스 삭제 함수
def drop_all_indexes(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop_indexes()

# 인덱스 정보 출력 함수
def print_index_information(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        indexes = collection.index_information()
        print(f"Indexes for {collection_name}: {indexes}")


# 쿼리 함수 정의
def q2(tag):
    # ml_tags에서 해당 tag를 가진 document의 movieId들 가져오기
    movie_ids = db.ml_tags.distinct("movieId", {"tag": tag})

    # ml_movies에서 해당 movieId들로 document의 title들 가져오기 (중복 없이 오름차순 정렬)
    titles = db.ml_movies.find({"movieId": {"$in": movie_ids}}, {"title": 1, "_id": 0}).sort("title", 1)
    unique_titles = sorted(set(doc["title"] for doc in titles))

    return unique_titles        


def q3(title):
    # ml_movies에서 입력받은 title로 검색한 document의 movieId 값을 찾기
    movie_id = db.ml_movies.find_one({'title': title}, {'movieId': 1})

    if movie_id:
        # 해당 movieId로 ml_ratings에서 해당 영화의 평점을 가져옵니다.
        ratings = [doc['rating'] for doc in db.ml_ratings.find({'movieId': movie_id['movieId']}, {'rating': 1})]

        # 평균 평점을 계산합니다.
        average_rating = sum(ratings) / len(ratings) if ratings else None
        return average_rating
    else:
        print("영화를 찾을 수 없습니다.")
        return None





def q4(userId):
    # 특정 userId가 평가한 영화의 movieId 목록 구하기
    movie_ids = db.ml_ratings.distinct("movieId", {"userId": userId})

    # userId가 평가한 영화의 개수와 각 영화의 평점과 평균 평점의 차이를 누적할 변수 초기화
    total_bias = 0
    num_rated_movies = len(movie_ids)

    if num_rated_movies > 0:
        # 평균 평점을 계산하기 위한 aggregate 파이프라인 정의
        pipeline = [
            {"$match": {"movieId": {"$in": movie_ids}}},
            {"$group": {"_id": "$movieId", "avgRating": {"$avg": "$rating"}}}
        ]

        # aggregate를 사용하여 평균 평점 계산
        avg_ratings = db.ml_ratings.aggregate(pipeline)

        # movieId별로 평균 평점을 딕셔너리에 저장
        movie_avg_ratings_dict = {rating['_id']: rating['avgRating'] for rating in avg_ratings}

        # userId로 제한하여 해당 사용자의 영화별 평점 구하기
        user_ratings_cursor = db.ml_ratings.find({'userId': userId}, {'movieId': 1, 'rating': 1})

        # 각 영화의 평점과 평균 평점의 차이를 계산하여 누적
        for user_rating in user_ratings_cursor:
            movie_id = user_rating['movieId']
            rating = user_rating['rating']
            avg_rating_by_movie = movie_avg_ratings_dict.get(movie_id, 0)
            total_bias += rating - avg_rating_by_movie

        # userId별 평균 평점의 바이어스를 반환
        return total_bias / num_rated_movies
    else:
        return 0








def q1():

    # 1. 인덱스 삭제
    drop_all_indexes(db)

    # 2. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (삭제 후):")
    print_index_information(db)
    print(end='\n')
        
    input_tag = "Gael Garcia Bernal"
    input_title = "Interstellar (2014)"
    input_userid = 54313

    before_time = []
    # 3. 코드 실행 결과 출력 및 생성 전 시간 저장
    results_pre = {}
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_pre[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_pre[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_pre[query_name]}")
        before_time.append(end_time - start_time)


    # 4. 인덱스 생성
    create_index(db.ml_tags, [('tag', 1)])
    create_index(db.ml_movies, [('movieId', 1), ('title', 1)])
    create_index(db.ml_ratings, [('movieId', 1), ('userId', 1)])

    # 5. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (생성 후):")
    print_index_information(db)    
    print(end='\n')

    # 6. 코드 실행 결과 출력 및 생성 후 시간 저장
    results_post = {}
    
    after_time = []
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_post[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_post[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_post[query_name]}")
        after_time.append(end_time - start_time)

    print()
    # 쿼리 실행 전 후 시간 차이 계산
    diffs = [b-f for b, f in zip(before_time, after_time)]
    print("\n쿼리 실행 전 후 시간 차이:")
    
    for i in range(0, 3):
        print('-'*20)
        print(f"Question{i+2}:")
        print(f"Index 생성 전 시간: {before_time[i]:.3f}초")
        print(f"Index 생성 후 시간: {after_time[i]:.3f}초")
        print(f"전 후 차이 시간: {diffs[i]:.3f}초")
        i =+1
    

        


    # 평균 차이값 계산
    print('-'*20)
    avg_diff = sum(diffs) / len(diffs)
    print(f"\n평균 차이값: {avg_diff:.3f}초")



# 예시 실행
q1()




Index 정보 (삭제 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}}

q2 실행 결과: ['Bad Education (La mala educación) (2004)', 'Mammoth (Mammut) (2009)']
q3 실행 결과: 4.097927896085535
q4 실행 결과: -0.9189320715108532
Index 정보 (생성 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}, 'movieId_1_title_1': {'v': 2, 'key': [('movieId', 1), ('title', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}, 'tag_1': {'v': 2, 'key': [('tag', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}, 'movieId_1_userId_1': {'v': 2, 'key': [('movieId', 1), ('userId', 1)], 'ns': 'movielens.ml_ratings'}}

q2 실행 결과

In [9]:
from pymongo import MongoClient
import time

# MongoDB 연결 설정
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']  # 데이터베이스 선택

# 인덱스 생성 함수
def create_index(collection, index_spec):
    if index_spec is not None:
        collection.create_index(index_spec)

# 인덱스 삭제 함수
def drop_all_indexes(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop_indexes()

# 인덱스 정보 출력 함수
def print_index_information(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        indexes = collection.index_information()
        print(f"Indexes for {collection_name}: {indexes}")


# 쿼리 함수 정의
def q2(tag):
    # ml_tags에서 해당 tag를 가진 document의 movieId들 가져오기
    movie_ids = db.ml_tags.distinct("movieId", {"tag": tag})

    # ml_movies에서 해당 movieId들로 document의 title들 가져오기 (중복 없이 오름차순 정렬)
    titles = db.ml_movies.find({"movieId": {"$in": movie_ids}}, {"title": 1, "_id": 0}).sort("title", 1)
    unique_titles = sorted(set(doc["title"] for doc in titles))

    return unique_titles        


def q3(title):
    # ml_movies에서 입력받은 title로 검색한 document의 movieId 값을 찾기
    movie_id = db.ml_movies.find_one({'title': title}, {'movieId': 1})

    if movie_id:
        # 해당 movieId로 ml_ratings에서 해당 영화의 평점을 가져옵니다.
        ratings = [doc['rating'] for doc in db.ml_ratings.find({'movieId': movie_id['movieId']}, {'rating': 1})]

        # 평균 평점을 계산합니다.
        average_rating = sum(ratings) / len(ratings) if ratings else None
        return average_rating
    else:
        print("영화를 찾을 수 없습니다.")
        return None




def q4(userId):
    # 특정 userId가 평가한 영화의 movieId 목록 구하기
    movie_ids = db.ml_ratings.distinct("movieId", {"userId": userId})

    # 특정 movieIds에 해당하는 문서에서 movieId, rating만 가져오기
    pipeline = [
        {"$match": {"movieId": {"$in": movie_ids}}},
        {"$group": {"_id": "$movieId", "avgRating": {"$avg": "$rating"}}}
    ]
    rating_aggregation = list(db.ml_ratings.aggregate(pipeline))

    # movieId별로 평균 평점을 저장할 딕셔너리 초기화
    movie_avg_ratings_dict = {rating['_id']: rating['avgRating'] for rating in rating_aggregation}

    # userId로 제한하여 해당 사용자의 영화별 평점 구하기
    user_ratings_cursor = db.ml_ratings.find({'userId': userId}, {'movieId': 1, 'rating': 1})

    # userId가 평가한 영화의 개수와 각 영화의 평점과 평균 평점의 차이를 누적할 변수 초기화
    total_bias = 0
    num_rated_movies = len(movie_ids)

    # 각 영화의 평점과 평균 평점의 차이를 계산하여 누적
    for user_rating in user_ratings_cursor:
        movie_id = user_rating['movieId']
        rating = user_rating['rating']
        avg_rating_by_movie = movie_avg_ratings_dict.get(movie_id, 0)
        total_bias += rating - avg_rating_by_movie

    # userId별 평균 평점의 바이어스를 반환
    return total_bias / num_rated_movies if num_rated_movies > 0 else 0









def q1():

    # 1. 인덱스 삭제
    drop_all_indexes(db)

    # 2. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (삭제 후):")
    print_index_information(db)
    print(end='\n')
        
    input_tag = "Gael Garcia Bernal"
    input_title = "Interstellar (2014)"
    input_userid = 121120

    before_time = []
    # 3. 코드 실행 결과 출력 및 생성 전 시간 저장
    results_pre = {}
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_pre[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_pre[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_pre[query_name]}")
        before_time.append(end_time - start_time)
    print(end_time - start_time)
    print(end='\n')

    # 4. 인덱스 생성
    create_index(db.ml_tags, [('tag', 1)])
    create_index(db.ml_movies, [('movieId', 1), ('title', 1)])
    create_index(db.ml_ratings, [('movieId', 1), ('userId', 1)])

    # 5. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (생성 후):")
    print_index_information(db)    
    print(end='\n')

    # 6. 코드 실행 결과 출력 및 생성 후 시간 저장
    results_post = {}
    
    after_time = []
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_post[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_post[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_post[query_name]}")
        after_time.append(end_time - start_time)
    print(end_time - start_time)
    print()
    # 쿼리 실행 전 후 시간 차이 계산
    diffs = [b-f for b, f in zip(before_time, after_time)]
    print("\n쿼리 실행 전 후 시간 차이:")
    
    for i in range(0, 3):
        print('-'*20)
        print(f"Question{i+2}:")
        print(f"Index 생성 전 시간: {before_time[i]:.3f}초")
        print(f"Index 생성 후 시간: {after_time[i]:.3f}초")
        print(f"전 후 차이 시간: {diffs[i]:.3f}초")
        i =+1
    

        


    # 평균 차이값 계산
    print('-'*20)
    avg_diff = sum(diffs) / len(diffs)
    print(f"\n평균 차이값: {avg_diff:.3f}초")



# 예시 실행
q1()




Index 정보 (삭제 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}}

q2 실행 결과: ['Bad Education (La mala educación) (2004)', 'Mammoth (Mammut) (2009)']
q3 실행 결과: 4.097927896085535
q4 실행 결과: -0.580230297564422
36.32050037384033

Index 정보 (생성 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}, 'movieId_1_userId_1': {'v': 2, 'key': [('movieId', 1), ('userId', 1)], 'ns': 'movielens.ml_ratings'}}

q2 실행 결과: ['Bad Education (La mala educación) (2004)', 'Mammoth (Mammut) (2009)']
q3 실행 결과: 4.097927896085535
q4 실행 결과: -0.580230297564422
25.63054203987121

In [10]:
from pymongo import MongoClient
import time

# MongoDB 연결 설정
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']  # 데이터베이스 선택

# 인덱스 생성 함수
def create_index(collection, index_spec):
    if index_spec is not None:
        collection.create_index(index_spec)

# 인덱스 삭제 함수
def drop_all_indexes(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop_indexes()

# 인덱스 정보 출력 함수
def print_index_information(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        indexes = collection.index_information()
        print(f"Indexes for {collection_name}: {indexes}")


# 쿼리 함수 정의
def q2(tag):
    # ml_tags에서 해당 tag를 가진 document의 movieId들 가져오기
    movie_ids = db.ml_tags.distinct("movieId", {"tag": tag})

    # ml_movies에서 해당 movieId들로 document의 title들 가져오기 (중복 없이 오름차순 정렬)
    titles = db.ml_movies.find({"movieId": {"$in": movie_ids}}, {"title": 1, "_id": 0}).sort("title", 1)
    unique_titles = sorted(set(doc["title"] for doc in titles))

    return unique_titles        


def q3(title):
    # ml_movies에서 입력받은 title로 검색한 document의 movieId 값을 찾기
    movie_id = db.ml_movies.find_one({'title': title}, {'movieId': 1})

    if movie_id:
        # 해당 movieId로 ml_ratings에서 해당 영화의 평점을 가져옵니다.
        ratings = [doc['rating'] for doc in db.ml_ratings.find({'movieId': movie_id['movieId']}, {'rating': 1})]

        # 평균 평점을 계산합니다.
        average_rating = sum(ratings) / len(ratings) if ratings else None
        return average_rating
    else:
        print("영화를 찾을 수 없습니다.")
        return None




def q4(userId):
    # 특정 userId가 평가한 영화의 movieId 목록 구하기
    movie_ids = db.ml_ratings.distinct("movieId", {"userId": userId})

    # movieId별로 평균 평점을 저장할 딕셔너리 초기화
    movie_avg_ratings_dict = {}

    # 평균 평점을 계산하기 위한 aggregate 파이프라인 정의
    pipeline = [
        {"$match": {"movieId": {"$in": movie_ids}}},
        {"$group": {"_id": "$movieId", "avgRating": {"$avg": "$rating"}}}
    ]

    # aggregate를 사용하여 평균 평점 계산
    rating_aggregation = list(db.ml_ratings.aggregate(pipeline))

    # movieId별로 평균 평점을 딕셔너리에 저장
    for rating in rating_aggregation:
        movie_avg_ratings_dict[rating['_id']] = rating['avgRating']

    # userId로 제한하여 해당 사용자의 영화별 평점 구하기
    user_ratings_cursor = db.ml_ratings.find({'userId': userId}, {'movieId': 1, 'rating': 1})

    # userId가 평가한 영화의 개수와 각 영화의 평점과 평균 평점의 차이를 누적할 변수 초기화
    total_bias = 0
    num_rated_movies = len(movie_ids)

    # 각 영화의 평점과 평균 평점의 차이를 계산하여 누적
    for user_rating in user_ratings_cursor:
        movie_id = user_rating['movieId']
        rating = user_rating['rating']
        avg_rating_by_movie = movie_avg_ratings_dict.get(movie_id, 0)
        total_bias += rating - avg_rating_by_movie

    # userId별 평균 평점의 바이어스를 반환
    return total_bias / num_rated_movies if num_rated_movies > 0 else 0










def q1():

    # 1. 인덱스 삭제
    drop_all_indexes(db)

    # 2. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (삭제 후):")
    print_index_information(db)
    print(end='\n')
        
    input_tag = "Gael Garcia Bernal"
    input_title = "Interstellar (2014)"
    input_userid = 121120

    before_time = []
    # 3. 코드 실행 결과 출력 및 생성 전 시간 저장
    results_pre = {}
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_pre[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_pre[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_pre[query_name]}")
        before_time.append(end_time - start_time)
    print(end_time - start_time)
    print(end='\n')

    # 4. 인덱스 생성
    create_index(db.ml_tags, [('tag', 1)])
    create_index(db.ml_movies, [('movieId', 1), ('title', 1)])
    create_index(db.ml_ratings, [('movieId', 1), ('userId', 1)])

    # 5. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (생성 후):")
    print_index_information(db)    
    print(end='\n')

    # 6. 코드 실행 결과 출력 및 생성 후 시간 저장
    results_post = {}
    
    after_time = []
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_post[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_post[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_post[query_name]}")
        after_time.append(end_time - start_time)
    print(end_time - start_time)
    print()
    # 쿼리 실행 전 후 시간 차이 계산
    diffs = [b-f for b, f in zip(before_time, after_time)]
    print("\n쿼리 실행 전 후 시간 차이:")
    
    for i in range(0, 3):
        print('-'*20)
        print(f"Question{i+2}:")
        print(f"Index 생성 전 시간: {before_time[i]:.3f}초")
        print(f"Index 생성 후 시간: {after_time[i]:.3f}초")
        print(f"전 후 차이 시간: {diffs[i]:.3f}초")
        i =+1
    

        


    # 평균 차이값 계산
    print('-'*20)
    avg_diff = sum(diffs) / len(diffs)
    print(f"\n평균 차이값: {avg_diff:.3f}초")



# 예시 실행
q1()




Index 정보 (삭제 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}}

q2 실행 결과: ['Bad Education (La mala educación) (2004)', 'Mammoth (Mammut) (2009)']
q3 실행 결과: 4.097927896085535
q4 실행 결과: -0.580230297564422
34.88946557044983

Index 정보 (생성 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}, 'movieId_1_title_1': {'v': 2, 'key': [('movieId', 1), ('title', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}, 'tag_1': {'v': 2, 'key': [('tag', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}, 'movieId_1_userId_1': {'v': 2, 'key': [('movieId', 1), ('userId', 1)], 'ns': 'movielens.ml_ra

In [11]:
from pymongo import MongoClient
import time

# MongoDB 연결 설정
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']  # 데이터베이스 선택

# 인덱스 생성 함수
def create_index(collection, index_spec):
    if index_spec is not None:
        collection.create_index(index_spec)

# 인덱스 삭제 함수
def drop_all_indexes(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop_indexes()

# 인덱스 정보 출력 함수
def print_index_information(db):
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        indexes = collection.index_information()
        print(f"Indexes for {collection_name}: {indexes}")


# 쿼리 함수 정의
def q2(tag):
    # ml_tags에서 해당 tag를 가진 document의 movieId들 가져오기
    movie_ids = db.ml_tags.distinct("movieId", {"tag": tag})

    # ml_movies에서 해당 movieId들로 document의 title들 가져오기 (중복 없이 오름차순 정렬)
    titles = db.ml_movies.find({"movieId": {"$in": movie_ids}}, {"title": 1, "_id": 0}).sort("title", 1)
    unique_titles = sorted(set(doc["title"] for doc in titles))

    return unique_titles        


def q3(title):
    # ml_movies에서 입력받은 title로 검색한 document의 movieId 값을 찾기
    movie_id = db.ml_movies.find_one({'title': title}, {'movieId': 1})

    if movie_id:
        # 해당 movieId로 ml_ratings에서 해당 영화의 평점을 가져옵니다.
        ratings = [doc['rating'] for doc in db.ml_ratings.find({'movieId': movie_id['movieId']}, {'rating': 1})]

        # 평균 평점을 계산합니다.
        average_rating = sum(ratings) / len(ratings) if ratings else None
        return average_rating
    else:
        print("영화를 찾을 수 없습니다.")
        return None





def q4(userId):
    # 특정 userId가 평가한 영화의 movieId 목록 구하기
    movie_ids = db.ml_ratings.distinct("movieId", {"userId": userId})

    # userId가 평가한 영화의 개수와 각 영화의 평점과 평균 평점의 차이를 누적할 변수 초기화
    total_bias = 0
    num_rated_movies = len(movie_ids)

    if num_rated_movies > 0:
        # 평균 평점을 계산하기 위한 aggregate 파이프라인 정의
        pipeline = [
            {"$match": {"movieId": {"$in": movie_ids}}},
            {"$group": {"_id": "$movieId", "avgRating": {"$avg": "$rating"}}}
        ]

        # aggregate를 사용하여 평균 평점 계산
        avg_ratings = db.ml_ratings.aggregate(pipeline)

        # movieId별로 평균 평점을 딕셔너리에 저장
        movie_avg_ratings_dict = {rating['_id']: rating['avgRating'] for rating in avg_ratings}

        # userId로 제한하여 해당 사용자의 영화별 평점 구하기
        user_ratings_cursor = db.ml_ratings.find({'userId': userId}, {'movieId': 1, 'rating': 1})

        # 각 영화의 평점과 평균 평점의 차이를 계산하여 누적
        for user_rating in user_ratings_cursor:
            movie_id = user_rating['movieId']
            rating = user_rating['rating']
            avg_rating_by_movie = movie_avg_ratings_dict.get(movie_id, 0)
            total_bias += rating - avg_rating_by_movie

        # userId별 평균 평점의 바이어스를 반환
        return total_bias / num_rated_movies
    else:
        return 0








def q1():

    # 1. 인덱스 삭제
    drop_all_indexes(db)

    # 2. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (삭제 후):")
    print_index_information(db)
    print(end='\n')
        
    input_tag = "Gael Garcia Bernal"
    input_title = "Interstellar (2014)"
    input_userid = 121120

    before_time = []
    # 3. 코드 실행 결과 출력 및 생성 전 시간 저장
    results_pre = {}
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_pre[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_pre[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_pre[query_name]}")
        before_time.append(end_time - start_time)
    print(end_time - start_time)        
    print(end='\n')

    # 4. 인덱스 생성
    create_index(db.ml_tags, [('tag', 1)])
    create_index(db.ml_movies, [('movieId', 1), ('title', 1)])
    create_index(db.ml_ratings, [('movieId', 1), ('userId', 1)])

    # 5. 인덱스 삭제 전 인덱스 정보 출력
    print("Index 정보 (생성 후):")
    print_index_information(db)    
    print(end='\n')

    # 6. 코드 실행 결과 출력 및 생성 후 시간 저장
    results_post = {}
    
    after_time = []
    for query_name, query_func, param in [('q2', q2, input_tag), ('q3', q3, input_title), ('q4', q4, input_userid)]:
        start_time = time.time()
        query_result = query_func(param)  # 함수 실행
        # 결과 처리 방식 변경
        if query_name == 'q3' or query_name == 'q4':  # q3, 4의 경우 단일 float 값을 직접 저장
            results_post[query_name] = query_result
        else:  # 다른 쿼리 결과(list)를 처리
            results_post[query_name] = list(query_result)
        end_time = time.time()
        print(f"{query_name} 실행 결과: {results_post[query_name]}")
        after_time.append(end_time - start_time)
    print(end_time - start_time)
    print()
    # 쿼리 실행 전 후 시간 차이 계산
    diffs = [b-f for b, f in zip(before_time, after_time)]
    print("\n쿼리 실행 전 후 시간 차이:")
    
    for i in range(0, 3):
        print('-'*20)
        print(f"Question{i+2}:")
        print(f"Index 생성 전 시간: {before_time[i]:.3f}초")
        print(f"Index 생성 후 시간: {after_time[i]:.3f}초")
        print(f"전 후 차이 시간: {diffs[i]:.3f}초")
        i =+1
    

        


    # 평균 차이값 계산
    print('-'*20)
    avg_diff = sum(diffs) / len(diffs)
    print(f"\n평균 차이값: {avg_diff:.3f}초")



# 예시 실행
q1()




Index 정보 (삭제 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}}

q2 실행 결과: ['Bad Education (La mala educación) (2004)', 'Mammoth (Mammut) (2009)']
q3 실행 결과: 4.097927896085535
q4 실행 결과: -0.580230297564422
35.992201805114746

Index 정보 (생성 후):
Indexes for ml_movies: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_movies'}, 'movieId_1_title_1': {'v': 2, 'key': [('movieId', 1), ('title', 1)], 'ns': 'movielens.ml_movies'}}
Indexes for ml_tags: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_tags'}, 'tag_1': {'v': 2, 'key': [('tag', 1)], 'ns': 'movielens.ml_tags'}}
Indexes for ml_ratings: {'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'movielens.ml_ratings'}, 'movieId_1_userId_1': {'v': 2, 'key': [('movieId', 1), ('userId', 1)], 'ns': 'movielens.ml_r

In [12]:



from sys import argv
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.movielens

def q4(input_userid: int):
    # TODO:# 특정 userId가 평가한 영화의 movieId 목록 구하기
    movie_ids = db.ml_ratings.distinct("movieId", {"userId": input_userid})

    # 특정 movieIds에 해당하는 문서에서 movieId, rating만 가져오기
    pipeline = [
        {"$match": {"movieId": {"$in": movie_ids}}},
        {"$group": {"_id": "$movieId", "avgRating": {"$avg": "$rating"}}}
    ]
    rating_aggregation = list(db.ml_ratings.aggregate(pipeline))

    # movieId별로 평균 평점을 저장할 딕셔너리 초기화
    movie_avg_ratings_dict = {rating['_id']: rating['avgRating'] for rating in rating_aggregation}

    # userId로 제한하여 해당 사용자의 영화별 평점 구하기
    user_ratings_cursor = db.ml_ratings.find({'userId': input_userid}, {'movieId': 1, 'rating': 1})

    # userId가 평가한 영화의 개수와 각 영화의 평점과 평균 평점의 차이를 누적할 변수 초기화
    total_bias = 0
    num_rated_movies = len(movie_ids)

    # 각 영화의 평점과 평균 평점의 차이를 계산하여 누적
    for user_rating in user_ratings_cursor:
        movie_id = user_rating['movieId']
        rating = user_rating['rating']
        avg_rating_by_movie = movie_avg_ratings_dict.get(movie_id, 0)
        total_bias += rating - avg_rating_by_movie

    # userId별 평균 평점의 바이어스를 반환
    total_bias / num_rated_movies if num_rated_movies > 0 else 0

    bias = total_bias / num_rated_movies if num_rated_movies > 0 else 0
    print('{:.3f}'.format(bias))

q4(62986)

# if __name__ == '__main__':
#     q4(int(argv[1]))

